# CSGO Demofile Processing
##### Peter Xenopoulos
##### January 31, 2020
This Jupyter notebook contains the code detailing how we process demofiles and create game state data.

In [ ]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import sqlite3

from csgo.parser.match_parser import CSGOMatchParser

# Parse damages
def parse_damages(df):
    """ Pass through df to create players remaining information
    """
    team_info_remaining = dict()
    team_info_remaining["CTHpRemaining"] = 500
    team_info_remaining["THpRemaining"] = 500
    team_info_remaining["CTRemaining"] = 5
    team_info_remaining["TRemaining"] = 5
    # Process
    tick_damages = df.groupby(["VictimID", "VictimSide"]).HpDamage.sum().reset_index()
    d = np.array(tick_damages["HpDamage"].values.tolist())
    tick_damages["HpDamage"] = np.where(d > 100, 100, d).tolist()
    # Personnel Remaining
    team_info_remaining["CTRemaining"] = 5 - tick_damages[(tick_damages["HpDamage"] == 100) & (tick_damages["VictimSide"] == "CT")].shape[0]
    team_info_remaining["TRemaining"] = 5 - tick_damages[(tick_damages["HpDamage"] == 100) & (tick_damages["VictimSide"] == "T")].shape[0]
    # HP Remaining
    total_damage = tick_damages.groupby("VictimSide").HpDamage.sum().reset_index()
    total_damage["HpLeft"] = 500 - total_damage["HpDamage"]
    if total_damage[total_damage["VictimSide"] == "CT"].shape[0] > 0:
        team_info_remaining["CTHpRemaining"] = total_damage[total_damage["VictimSide"] == "CT"].HpLeft.values[0]
    if total_damage[total_damage["VictimSide"] == "T"].shape[0] > 0:
        team_info_remaining["THpRemaining"] = total_damage[total_damage["VictimSide"] == "T"].HpLeft.values[0]
    players_killed = tick_damages[tick_damages["HpDamage"] == 100].VictimID.values
    return team_info_remaining, players_killed

def create_game_state(tick, start_tick, round_damages, round_footsteps, round_bomb_events, row, round, _ct_win, _ct_eq_val, _t_eq_val):
    """ Create game state
    """
    _time_since_start = tick - start_tick
    # Determine hp of each team and people alive
    team_info, players_killed = parse_damages(round_damages[round_damages["Tick"] < tick])
    _t_hp_left = team_info["THpRemaining"]
    _ct_hp_left = team_info["CTHpRemaining"]
    _t_left = team_info["TRemaining"]
    _ct_left = team_info["CTRemaining"]
    # Get alive players
    alive_players = [p for p in players if p not in players_killed]
    filtered_footsteps_ct = round_footsteps[(round_footsteps["Tick"] <= tick) & (round_footsteps["SteamID"].isin(alive_players)) & (round_footsteps["Side"] == "CT")]
    filtered_footsteps_t = round_footsteps[(round_footsteps["Tick"] <= tick) & (round_footsteps["SteamID"].isin(alive_players)) & (round_footsteps["Side"] == "T")]
    last_ct_positions = filtered_footsteps_ct.loc[filtered_footsteps_ct.groupby("SteamID").Tick.idxmax()]
    last_t_positions = filtered_footsteps_t.loc[filtered_footsteps_t.groupby("SteamID").Tick.idxmax()]
    _min_ct_a = last_ct_positions.DistanceBombsiteA.min()
    _min_ct_b = last_ct_positions.DistanceBombsiteB.min()
    _min_t_a = last_t_positions.DistanceBombsiteA.min()
    _min_t_b = last_t_positions.DistanceBombsiteB.min()
    # Determine if bomb is planted
    _bomb_planted = 0
    _bomb_site = "NotPlanted"
    if round_bomb_events[round_bomb_events["Tick"] < tick].shape[0] == 1:
        _bomb_planted = 1
        _bomb_site = round_bomb_events.iloc[0,9]
    damage_event = False
    if round_damages[round_damages["Tick"] == tick].shape[0] == 1:
        damage_event = True
    # Frame of data
    tick_frame = dict()
    tick_frame["CompetitionName"] = row["CompetitionName"]
    tick_frame["MatchName"] = row["MatchName"]
    tick_frame["GameDate"] = row["GameDate"]
    tick_frame["GameTime"] = row["GameTime"]
    tick_frame["MapName"] = round["MapName"]
    tick_frame["RoundNum"] = round["RoundNum"]
    tick_frame["Tick"] = tick
    tick_frame["TicksSinceStart"] = _time_since_start
    tick_frame["CTWin"] = _ct_win
    tick_frame["CTEqVal"] = _ct_eq_val
    tick_frame["TEqVal"]  = _t_eq_val
    tick_frame["TRemaining"] = _t_left
    tick_frame["CTRemaining"] = _ct_left
    tick_frame["THpRemaining"] = _t_hp_left
    tick_frame["CTHpRemaining"] = _ct_hp_left
    tick_frame["BombPlanted"] = _bomb_planted
    tick_frame["BombSite"] = _bomb_site
    tick_frame["CTDistBombsiteA"] = _min_ct_a
    tick_frame["CTDistBombsiteB"] = _min_ct_b
    tick_frame["TDistBombsiteA"] = _min_t_a
    tick_frame["TDistBombsiteB"] = _min_t_b
    if damage_event:
        tick_frame["AttackerID"] = round_damages[round_damages["Tick"] == tick].AttackerID.values[0]
        tick_frame["AttackerName"] = round_damages[round_damages["Tick"] == tick].AttackerName.values[0]
        tick_frame["AttackerTeam"] = round_damages[round_damages["Tick"] == tick].AttackerTeam.values[0]
        tick_frame["VictimID"] = round_damages[round_damages["Tick"] == tick].VictimID.values[0]
        tick_frame["VictimName"] = round_damages[round_damages["Tick"] == tick].VictimName.values[0]
        tick_frame["VictimTeam"] = round_damages[round_damages["Tick"] == tick].VictimTeam.values[0]
    else:
        tick_frame["AttackerID"] = np.nan
        tick_frame["AttackerName"] = np.nan
        tick_frame["AttackerTeam"] = np.nan
        tick_frame["VictimID"] = np.nan
        tick_frame["VictimName"] = np.nan
        tick_frame["VictimTeam"] = np.nan
    return tick_frame

# Parse the demofile and output to data/example_game_states.csv
p = CSGOMatchParser(demofile = "data/test.dem", logfile = "logs/parser.log", competition_name = "TestCompetition", match_name = "MatchName")
try:
    p.parse_demofile()
    # If error occured, don't parse the file
    if p.demo_error:
        pass
    else:
        p.find_match_start()
        p.parse_match()
        p.write_rounds()
        p.write_kills()
        p.write_damages()
        p.write_bomb_events()
        p.write_footsteps()
        rounds = p.rounds_df
        kills = p.kills_df
        damages = p.damages_df
        bomb_events = p.bomb_df
        footsteps = p.footsteps_df
        for j, round in rounds.iterrows():
            output = pd.DataFrame()
            # Get Match Metadata
            round_num = round["RoundNum"]
            win_reason = round["Reason"]
            start_tick = round["StartTick"]
            if round["RoundWinnerSide"] == "CT":
                _ct_win = 1
            else:
                _ct_win = 0
            ticks = []
            # Footsteps
            round_footsteps = footsteps[footsteps["RoundNum"] == round_num]
            try:
                start_tick = round_footsteps.Tick.min()
            except:
                pass
            starting_tick = round_footsteps.groupby(["SteamID"]).Tick.min().max()
            # Bomb Events
            round_bomb_events = bomb_events[bomb_events["RoundNum"] == round_num]
            # Damage Parsing
            round_damages = damages[damages["RoundNum"] == round_num]
            try:
                _ct_eq_val = round_damages[round_damages["AttackerSide"] == "CT"].AttackerTeamEqVal.values[0]
            except:
                _ct_eq_val = np.nan
            try:
                _t_eq_val = round_damages[round_damages["AttackerSide"] == "T"].AttackerTeamEqVal.values[0]
            except:
                _t_eq_val = np.nan
            # Fix round damages
            round_damages["Tick"] += round_damages.groupby("Tick").cumcount() / round_damages.groupby("Tick").size().max()
            ticks.extend(round_footsteps.Tick.values)
            ticks.extend(round_bomb_events.Tick.values)
            ticks.extend(round_damages.Tick.values)
            ticks = list(set(ticks))
            ticks = [t for t in ticks if t >= starting_tick]
            ticks.sort()
            players = round_footsteps["SteamID"].unique()
            pool = mp.Pool(4)
            output = pool.starmap(create_tick_slice, [(tick, start_tick, round_damages, round_footsteps, round_bomb_events, row, round, _ct_win, _ct_eq_val, _t_eq_val) for tick in ticks]) 
            pool.close()
            output = pd.DataFrame(output)
            output.to_csv("data/example_game_states.csv", index=False)
except:
    pass